In [1]:
# Regra geral para determinar o K = Sqrt(n) é o padrão depois realize testes
# onde "n" é a quantidade de exemplos de treinamento
# K muito pequeno é sucetivel a qualquer ruido
# K muito grande torna o processo computacionalmente caro
# Essa abortagem de K = Sqrt(n) é um padrão

In [2]:
'''
Implementação do kNN
dataset: https://archive.ics.uci.edu/ml/machine-learning-databases/haberman/haberman.data
Descrição: Sobrevivencia de pacientes submetidos a cirurgia de mama
'''

# lista de amostras
amostras =[]

In [3]:
with open('dataset.data','r') as f:
    for linha in f.readlines():
        atrib = linha.replace('\n','').split(',')
        amostras.append([int(atrib[0]),int(atrib[1]),
                        int(atrib[2]),int(atrib[3])])

In [4]:
amostras

[[30, 64, 1, 1],
 [30, 62, 3, 1],
 [30, 65, 0, 1],
 [31, 59, 2, 1],
 [31, 65, 4, 1],
 [33, 58, 10, 1],
 [33, 60, 0, 1],
 [34, 59, 0, 2],
 [34, 66, 9, 2],
 [34, 58, 30, 1],
 [34, 60, 1, 1],
 [34, 61, 10, 1],
 [34, 67, 7, 1],
 [34, 60, 0, 1],
 [35, 64, 13, 1],
 [35, 63, 0, 1],
 [36, 60, 1, 1],
 [36, 69, 0, 1],
 [37, 60, 0, 1],
 [37, 63, 0, 1],
 [37, 58, 0, 1],
 [37, 59, 6, 1],
 [37, 60, 15, 1],
 [37, 63, 0, 1],
 [38, 69, 21, 2],
 [38, 59, 2, 1],
 [38, 60, 0, 1],
 [38, 60, 0, 1],
 [38, 62, 3, 1],
 [38, 64, 1, 1],
 [38, 66, 0, 1],
 [38, 66, 11, 1],
 [38, 60, 1, 1],
 [38, 67, 5, 1],
 [39, 66, 0, 2],
 [39, 63, 0, 1],
 [39, 67, 0, 1],
 [39, 58, 0, 1],
 [39, 59, 2, 1],
 [39, 63, 4, 1],
 [40, 58, 2, 1],
 [40, 58, 0, 1],
 [40, 65, 0, 1],
 [41, 60, 23, 2],
 [41, 64, 0, 2],
 [41, 67, 0, 2],
 [41, 58, 0, 1],
 [41, 59, 8, 1],
 [41, 59, 0, 1],
 [41, 64, 0, 1],
 [41, 69, 8, 1],
 [41, 65, 0, 1],
 [41, 65, 0, 1],
 [42, 69, 1, 2],
 [42, 59, 0, 2],
 [42, 58, 0, 1],
 [42, 60, 1, 1],
 [42, 59, 2, 1],
 [42, 

In [5]:
len(amostras)

306

In [12]:
def info_data_set(amostras, verbose=True):
    if verbose:
        print('Total de amostras: %d' % len(amostras))
    rotulo1, rotulo2 = 0, 0
    for amostra in amostras:
        if amostra[-1] == 1:
            rotulo1 += 1
        else:
            rotulo2 += 1
    if verbose:
        print('Total rotulo 1: %d' % rotulo1)
        print('Total rotulo 2: %d' % rotulo2)
    return [len(amostras),rotulo1,rotulo2]

In [13]:
info_data_set(amostras)

Total de amostras: 306
Total rotulo 1: 225
Total rotulo 2: 81


[306, 225, 81]

In [14]:
_, rotulo1, rotulo2 = info_data_set(amostras,verbose=False) #capturando o valor de cada variado passado na função, "_," pega nada

In [15]:
p = 0.6 # % que iremos utilizar da cada amostra
treinamento, teste = [], [] # criando duas listas uma para treinamento outra para teste
max_rotulo1, max_rotulo2 = int(p * rotulo1), int(p * rotulo2) #multiplicamos para saber o numero maximo que podemos utilizar para o treinamento

In [16]:
total_rotulo1, total_rotulo2 = 0, 0
for amostra in amostras:
    if(total_rotulo1 + total_rotulo2) < (max_rotulo1 + max_rotulo2):
        treinamento.append(amostra)
        if amostra[-1] == 1 and total_rotulo1 < max_rotulo1:
            total_rotulo1 += 1
        else:
            total_rotulo2 += 1
    else:
        teste.append(amostra)

In [17]:
print(len(treinamento))
print(len(teste))
info_data_set(treinamento)

183
123
Total de amostras: 183
Total rotulo 1: 132
Total rotulo 2: 51


[183, 132, 51]

In [18]:
info_data_set(teste)

Total de amostras: 123
Total rotulo 1: 93
Total rotulo 2: 30


[123, 93, 30]

In [19]:
import math

In [20]:
def dist_euclidiana(v1, v2):
    dim = len(v1)
    soma = 0
    for i in range(dim - 1):#Não queremos pegar o atributo de saida
        soma += math.pow(v1[i] - v2[i],2)
    return math.sqrt(soma)

In [22]:
# teste distancia
v1 = [2,1,3]
v2 = [2,1,3]
dist_euclidiana(v1,v2)

0.0

In [56]:
def knn(treinamento, nova_amostra, K):
    tam_treino = len(treinamento)
    dists = {}
    # Calculo da distancia da nova amostra para todos os outros 
    #exemplos
    for i in range(tam_treino):
        d = dist_euclidiana(treinamento[i], nova_amostra)
        dists[i] = d
        
    # Obtem as chaves dos k vizinhos mais proximos
    k_vizinhos = sorted(dists, key= dists.get)[:K]
    
    # Votação majoritaria
    qtd_rotulo1 = 0
    qtd_rotulo2 = 0
    for indice in k_vizinhos:
        if treinamento[indice][-1] == 1:
            qtd_rotulo1 += 1
        else:
            qtd_rotulo2 += 1
            
    if qtd_rotulo1 > qtd_rotulo2:
        return 1
    else:
        return 2

In [43]:
print(teste[30]) #Amostra
print(knn(treinamento, teste[30], K= 4)) #Teste de verdade

[58, 58, 3, 1]
1


In [75]:
acertos = 0
K = 42
for amostra in teste:
    classe = knn(treinamento, amostra, K)
    if amostra[-1] == classe:
        acertos += 1

print('Total de Treinamento %d' % len(treinamento))
print('Total de testes %d'  % len(teste))
print('Total de acertos %d' % acertos )
print( 'Total de acertos: %.2f%%' % (100 * acertos / len(teste)))

Total de Treinamento 183
Total de testes 123
Total de acertos 96
Total de acertos: 78.05%
